In [1]:
import datetime
import pandas as pd
import numpy as np
import scipy as sp
from sklearn.model_selection import train_test_split
from sklearn.metrics import average_precision_score
from catboost import CatBoostClassifier, Pool
from sklearn.model_selection import GridSearchCV

In [4]:
from sklearn.metrics import f1_score
from sklearn.model_selection import cross_val_score
import datetime
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import average_precision_score
from catboost import CatBoostClassifier, Pool
from sklearn.model_selection import GridSearchCV
import numpy as np
from scipy.stats import randint
from catboost import CatBoostClassifier
from sklearn.model_selection import RandomizedSearchCV
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import uniform as sp_randFloat
from scipy.stats import randint as sp_randInt
from sklearn.metrics import f1_score, make_scorer

f1 = make_scorer(f1_score , average='micro')

#  {'depth': 5, 'iterations': 500, 'learning_rate': 0.01}
print(" CATBOOST ".center(80, "="))

print("Loading dataset...")

df = pd.read_csv("/Users/pryda/Documents/Projets_Jedha/2_ConversioNRate/conversion_data_train.csv")

X = df.drop(["converted"], axis=1)
y = df["converted"]

X_train, X_test, Y_train, Y_test = train_test_split(X, y, test_size=0.2, random_state=0, stratify=y)

categoricals = ["country", "new_user", "source"]

# model = CatBoostClassifier(iterations=500, learning_rate=0.01, depth=5, loss_function="Logloss", eval_metric="F1", cat_features=categoricals, verbose=100) # poop

# model = CatBoostClassifier(loss_function="Logloss", eval_metric="F1", cat_features=categoricals, verbose=100) # 0.777 train, 0.7517 test

# model = CatBoostClassifier(loss_function="Logloss", eval_metric="F1", cat_features=categoricals, verbose=100, depth=9, learning_rate=0.2, rsm=1) # f1-score on train set :  0.7910958904109588
# f1-score on test set :  0.7397585592717197

cbc = CatBoostClassifier(cat_features=categoricals, eval_metric="F1", verbose=100)
param_dist = {
    # "learning_rate": np.linspace(0,0.2,5),
    # "depth": randint(3, 10),
    # "iterations": randint(100, 500),
    "rsm": np.linspace(0,1),
    "learning_rate": sp_randFloat(0.01, 0.8),
    "depth": sp_randInt(6, 10),
    "iterations": sp_randInt(10, 1000),
    "border_count": sp_randInt(1, 255),
    'l2_leaf_reg': sp_randInt(1, 10),
    'loss_function': ['Logloss', 'CrossEntropy'],
}

model = RandomizedSearchCV(cbc, param_distributions=param_dist, n_iter=10, cv=10, scoring=f1, n_jobs=-1)

model.fit(X_train, Y_train, eval_set=(X_test, Y_test))

Y_train_pred = model.predict(X_train)
Y_test_pred = model.predict(X_test)

# Print the tuned parameters and score
print(model.best_params_)
print(model.best_score_)

print("f1-score on train set : ", f1_score(Y_train, Y_train_pred))
print("f1-score on test set : ", f1_score(Y_test, Y_test_pred))

# model.save_model("catboost_v1_randomsearch")

=================================== CATBOOST ===================================
Loading dataset...
0:	learn: 0.7120397	test: 0.7005838	best: 0.7005838 (0)	total: 87ms	remaining: 13.4s
100:	learn: 0.7692654	test: 0.7527720	best: 0.7543335 (38)	total: 887ms	remaining: 474ms
154:	learn: 0.7704391	test: 0.7514970	best: 0.7553033 (108)	total: 1.29s	remaining: 0us

bestTest = 0.7553032566
bestIteration = 108

Shrink model to first 109 iterations.
{'border_count': 160, 'depth': 6, 'iterations': 155, 'l2_leaf_reg': 4, 'learning_rate': 0.6346097877762438, 'loss_function': 'CrossEntropy', 'rsm': 0.2857142857142857}
0.9862648520676641
f1-score on train set :  0.7698400836945148
f1-score on test set :  0.7553032566477442


In [ ]:
# Concatenate our train and test set to train your best classifier on all data with labels
X = np.append(X_train,X_test,axis=0)
Y = np.append(Y_train,Y_test)

model.fit(X,Y)

In [ ]:
df_test = pd.read_csv("/Users/pryda/Documents/Projets_Jedha/2_ConversioNRate/conversion_data_test.csv")

predictions = model.predict(df_test)
print(predictions)

pred_df = pd.DataFrame(columns=['converted'], data=predictions)
# pred_df = pd.DataFrame({ "converted": np.array(predictions)[:, -1] })
pred_df.to_csv(f"conversion_data_test_predictions_LV_catboost_split{datetime.datetime.now().timestamp()}.csv",
               index=False)